# Word Count in Spark

### Setup

Spark 세팅

In [ ]:
# !pip install pyspark
# !pip install -U -q PyDrive
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Google Drive client 권한을 받아 Spark 실습을 위해 사용될 파일을 받기

In [ ]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
# downloaded = drive.CreateFile({'id': id})
# downloaded.GetContentFile('pg100.txt')

### 실습

*pg100.txt* 파일은 셰익스피어의 작품들의 사본들을 가지고 있습니다.

Spark를 사용하여 각 영어 letter로 시작하는 단어들의 갯수들을 출력하는 프로그램을 만들어보세요. 한마디로 각 letter에 대해, (non-unique)한 단어들의 총 갯수들을 세는 것입니다.

**대소문자는 무시하세요**, i.e. 단어들 전부 lower case라 생각하시면 됩니다. 또한, 알파벳이 아닌 character로 시작하는 단어들도 무시해도 됩니다. **문서 전체**에 대해 word count를 출력하세요(제목, 저자, 줄거리 포함). 라인 바꿈으로 -으로 연결된 단어들은 두 개의 다른 단어로 counting되어도 됩니다(e.g. "project", "pro-ject" 는 다른 단어).

출력에 대한 오차범위는 5미만으로 나오게 해주세요!

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [10]:
documents = sc.textFile('./pg100.txt')

counts = documents.flatMap(lambda line: line.split()).filter(lambda word: word.isalpha()) \
                        .map(lambda word: word[0].lower()) \
                        .map(lambda letter: (letter, 1)) \
                        .reduceByKey(lambda n1, n2: n1 + n2)\
                        .sortByKey(lambda letter_cnt:letter_cnt[0])
counts.toDF(['letter','count']).show(26)
sc.stop()

+------+------+
|letter| count|
+------+------+
|     a| 76666|
|     b| 35712|
|     c| 19564|
|     d| 20314|
|     e| 12056|
|     f| 27244|
|     g| 14405|
|     h| 48531|
|     i| 56010|
|     j|  1539|
|     k|  6314|
|     l| 18669|
|     m| 42047|
|     n| 20434|
|     o| 37408|
|     p| 16220|
|     q|  1428|
|     r|  8626|
|     s| 48269|
|     t|114500|
|     u|  7049|
|     v|  3442|
|     w| 49993|
|     x|     1|
|     y| 21646|
|     z|    33|
+------+------+
only showing top 26 rows

